## Script de travail sur le dataset feel
Ce script a pour but d'étudier le dataset feel, en terme de traitement NLP, de vectorisation, d'apprentissage de modèles de machine learning et leur impact sur le dataset de test.

#### Importation des librairies utiles

In [1]:
# Pour le traitement de données
import pandas as pd
import numpy as np

# Pour choisir les modèles de ML
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Pour gérer les fichiers et évaluer la performance temporelles
import sys
import os
import time
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), ".."))

# Scripts permettant de traiter les données
from code_project.data_science.utils_project import load_saved_file, read_feel, process_label, process_nlp, vectorize, apply_vectorization
from code_project.data_science.evaluate import load_test_set, evaluate_model
from code_project.config_project import *

pd.set_option('display.max_colwidth', -1)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


#### Overwrite du fichier de configuration
On écrit sur des données nouvelles pour éviter de casser les modèles pré-entrainés.

In [2]:
TYPE_VECTOR = "tf-idf"
VECTOR_NAME = "test_2.sav"
MODEL_NAME = "model_2.sav"

SAVE_DIR = os.path.join(os.path.dirname(CWD_PATH), "data", "modeles")
MODEL_PATH = os.path.join(SAVE_DIR, MODEL_NAME)
VECTOR_PATH = os.path.join(SAVE_DIR, VECTOR_NAME)

#### Options permettant de gérer le traitement des données ou la prédiction
- PREDICT_WORD : permet de prédire sur chaque mot (plutôt que sur chaque phrase). On agrège ensuite les prédictions à la phrase en récupérant le sentiment le plus présent.
- UP_SAMPLING : les classes des données étant déséquilibrées, on les ré-équilibre en augmentant le nombre de labels des classes moins présentes.
- ONE_VS_ALL : on prédit sur chaque classe et on prend la classe ayant la probabilité la plus grande plutôt que chercher à prédire la classe directement.

In [3]:
PREDICT_WORD = True
UP_SAMPLING = True
ONE_VS_ALL = False

#### Lecture des données
Lecture du dataset FEEL et des données de test

In [4]:
# Lecture des données
t1 = time.time()
feel = read_feel(FEEL_PATH, limit=LIMIT)
data_test = load_test_set(TEST_SET_PATH)
t2 = time.time()
print("Files read, duration : {}s".format(round(t2-t1, 4)))

Files read, duration : 0.0766s


#### Traitement des données
- On traite en premier les labels pour dédoubler les mots pouvant avoir plusieurs émotions
- On effectue un traitement NLP sur les phrases (tokenisation, normalisation, stemming)

In [5]:
t3 = time.time()
feel = process_label(feel)
feel['nlp_sentence'] = process_nlp(feel['sentence'])
data_test['nlp_sentence'] = process_nlp(data_test['phrase'])
t4 = time.time()
print("Data processed, duration : {}s".format(round(t4-t3, 4)))

Data processed, duration : 62.0088s


#### Séparation des données en features + label

In [6]:
x_train, y_train = feel.nlp_sentence, feel.emotion
x_test, y_test = data_test.nlp_sentence, data_test.emotion

#### Vectorisation des données
Suivant la méthode de vectorisation choisie on vectorise les mots : 
- CountVectorizer
- TfIdf
- Word2Vec

In [7]:
t5 = time.time()
feat_train, vectorizer = vectorize(x_train, type_vector=TYPE_VECTOR)
t6 = time.time()
print("Text vectorized, duration : {}s".format(round(t6-t5, 4)))

Text vectorized, duration : 0.1883s


#### Resample des données
Les classes sont déséquilibrées ici, on rajoute des labels dans les classes les moins présentes pour éviter l'overtfitting. La méthode choisie après plusieurs essais est celle de rajouter aléatoirement des données.

In [8]:
if UP_SAMPLING:
    from imblearn.over_sampling import RandomOverSampler
    t7 = time.time()
    upsamp = RandomOverSampler(random_state=777)
    
    feat_train, y_train = upsamp.fit_sample(feat_train, y_train)
    t8 = time.time()
    
    print(pd.DataFrame({'emotion' : y_train}).emotion.value_counts())
    print("Up-sampling done, duration : {}".format(t8-t7))

sadness       7912
disgust       7912
joy           7912
surprise      7912
anger         7912
no_emotion    7912
fear          7912
Name: emotion, dtype: int64
Up-sampling done, duration : 0.0404207706451416


### Modèles de machine learning
Après plusieurs essais, les 3 modèles les plus probants sont le RandomForest, le SVM et la régression logistique.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

rand_for = RandomForestClassifier(n_jobs=-1, n_estimators=100)
svc = SVC()
log_reg = LogisticRegression(n_jobs=-1)

#### Définition de la fonction de prédiction
Adaptation de la fonction de prédiction de chaque phrases dans le cas où l'option PREDICT_WORD est validée : on prédit sur chaque mot et on agrège les prédictions en conservant l'émotion la plus présente.

In [10]:
def most_common(lst):
    lst = [el for el in lst if el != 'no_emotion']
    if len(lst) == 0:
        return 'no_emotion'
    else:
        return max(set(lst), key=lst.count)
    
def predict(x_test, vectorizer, model, predict_word):
    
    predictions = []
    
    for sentence in x_test:
        if predict_word:
            list_emotion_word = []
            for word in sentence:
                feat_word = vectorizer.transform([word])
                emotion_word = model.predict(feat_word)
                list_emotion_word.append(list(emotion_word)[0])
            predictions.append(most_common(list_emotion_word))
        else:
            feat_sent = vectorizer.transform(sentence)
            emotion_sent = model.predict(feat_sent)
            predictions.append(list(emotion_sent)[0])
    
    return predictions

#### Sélection du meilleur modèle
A l'image du grid_search, cette fonction permet d'entrainer plusieurs modèles et de ne garder uniquement celui qui possède le score le plus grand.

In [13]:
def solve_grid(model, vectorizer, feat_train, y_train, x_test, y_test, param_grid, predict_word):
    
    grid = ParameterGrid(param_grid)
    print("{} models to train\n".format(len(grid)))
    
    best_score, best_params = 0, None
    
    for params in grid:
        print('model to train : \n-params {}'.format(params))
        # On change les paramètres et on fit le modèle à chaque itération
        model.set_params(**params)
        model.fit(feat_train, y_train)
        
        # On prédit sur les données de test
        predictions = predict(x_test, vectorizer, model, predict_word)
        score = accuracy_score(y_test, predictions)
        mat_conf = confusion_matrix(y_test, predictions)
        
        if score > best_score:
            best_score = score
            best_params = params
        
        print('-score {}'.format(score))
        print('confusion matrix : \n{}'.format(mat_conf))
    
    print("\nBEST_SCORE : {}".format(best_score))
    print("BEST_PARAMS : {}".format(best_params))
        
    return best_score, best_params

#### Apprentissage des différents modèles

In [14]:
# On conserve les paramètres optimisés ici pour une nouvelle passe plus fine de la fonction
params_rand_for = {'bootstrap': False, 'max_features': 'auto', 
                   'min_samples_leaf': 1, 'min_samples_split': 2, 
                   'n_estimators': 100}
params_svc = {'C': 5, 'gamma': 0.3, 'kernel': 'linear'}
params_log_reg = {'C': 5, 'multi_class': 'auto', 'penalty': 'l1'}

In [22]:
# Amélioration du random forest
param_grid = {
    'bootstrap': [True, False],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'n_estimators': [100, 200]
}

score_rand_for, params_rand_for = solve_grid(rand_for, vectorizer, feat_train, y_train, 
                                             x_test, y_test, param_grid, predict_word=PREDICT_WORD)

32 models to train

model to train : 
-params {'bootstrap': True, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


KeyboardInterrupt: 

In [ ]:
# Amélioration du SVM

param_grid = {
    'C': [4, 6],
    'gamma': [0.3, 0.4],
    'kernel': ['linear', 'sigmoid']
}

score_svc, params_svc = solve_grid(svc, vectorizer, feat_train, y_train, 
                                   x_test, y_test, param_grid, predict_word=PREDICT_WORD)

8 models to train

model to train : 
-params {'C': 4, 'gamma': 0.3, 'kernel': 'linear'}


In [22]:
# Amélioration de la régression logistique

param_grid = {
    "C": [1, 3, 5, 6], 
    "penalty": ["l1","l2"],
    "multi_class": ["auto", "ovr"]
}

score_log_reg, params_log_reg = solve_grid(log_reg, vectorizer, feat_train, y_train, 
                                           x_test, y_test, param_grid, predict_word=PREDICT_WORD)

14 models to train

model to train : 
-params {'C': 2, 'multi_class': 'auto', 'penalty': 'l1'}
['sadness', 'sadness', 'joy']
-score 0.2875816993464052
confusion matrix : 
[[ 7  2  2  2  3  3  5]
 [ 9  9  1  2  2  4  2]
 [ 5  1  5  5  1  1  5]
 [ 5  1  5  5  4  3 14]
 [ 0  0  0  0  0  0  0]
 [ 1  2  1  2  0 11  2]
 [ 5  2  1  4  0  2  7]]
model to train : 
-params {'C': 2, 'multi_class': 'auto', 'penalty': 'l2'}
['sadness', 'sadness', 'joy']
-score 0.28104575163398693
confusion matrix : 
[[ 7  3  1  2  3  4  4]
 [10 10  1  2  2  2  2]
 [ 4  1  5  5  1  2  5]
 [ 6  2  4  4  4  3 14]
 [ 0  0  0  0  0  0  0]
 [ 1  4  1  1  0 10  2]
 [ 4  2  2  4  0  2  7]]
model to train : 
-params {'C': 4, 'multi_class': 'auto', 'penalty': 'l1'}
['sadness', 'joy', 'joy']
-score 0.2875816993464052
confusion matrix : 
[[ 8  2  1  3  3  3  4]
 [10  9  0  2  2  4  2]
 [ 5  1  5  7  1  1  3]
 [ 6  0  4  8  4  3 12]
 [ 0  0  0  0  0  0  0]
 [ 1  2  1  4  0  9  2]
 [ 5  2  1  6  0  2  5]]
model to train : 
-para

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


['disgust', 'disgust', 'joy']
-score 0.2549019607843137
confusion matrix : 
[[ 0 19  1  2  0  1  1]
 [ 2 25  0  0  0  2  0]
 [ 2 15  1  3  0  0  2]
 [ 1 27  2  3  1  1  2]
 [ 0  0  0  0  0  0  0]
 [ 0 10  0  2  0  6  1]
 [ 3 11  0  2  0  1  4]]
model to train : 
-params {'C': 5, 'multi_class': 'auto', 'penalty': 'l2'}
['sadness', 'sadness', 'joy']
-score 0.27450980392156865
confusion matrix : 
[[ 7  3  1  2  3  4  4]
 [10 10  1  2  2  2  2]
 [ 5  1  4  5  1  2  5]
 [ 6  2  4  4  4  3 14]
 [ 0  0  0  0  0  0  0]
 [ 1  4  1  1  0 10  2]
 [ 4  2  2  4  0  2  7]]
model to train : 
-params {'C': 8, 'multi_class': 'auto', 'penalty': 'l1'}
['disgust', 'disgust', 'joy']
-score 0.24183006535947713
confusion matrix : 
[[ 0 18  1  2  0  2  1]
 [ 2 24  0  1  0  2  0]
 [ 2 14  1  3  0  0  3]
 [ 1 26  2  2  1  1  4]
 [ 0  0  0  0  0  0  0]
 [ 0 11  0  2  0  5  1]
 [ 3  9  0  2  0  2  5]]
model to train : 
-params {'C': 8, 'multi_class': 'auto', 'penalty': 'l2'}
['sadness', 'sadness', 'joy']
-score 0

#### Entrainement du modèle de bagging
On se sert des 3 modèles optimisés ci-dessus afin de réaliser un bagging. 

In [15]:
# Apprentissage des modèles optimisés
rand_for_opt = RandomForestClassifier(n_jobs=-1, **params_rand_for)
svc_opt = SVC(probability=True,**params_svc)
log_reg_opt = LogisticRegression(n_jobs=-1, **params_log_reg)

In [16]:
# Bagging des modèles ci-dessus
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(estimators=[('rl', log_reg_opt),('svm', svc_opt),('rf', rand_for_opt)],
                          voting='soft', weights = [1, 1, 1], n_jobs=-1)
voting.fit(feat_train, y_train)

VotingClassifier(estimators=[('rl', LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto', n_jobs=-1,
          penalty='l1', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)), ('svm', SVC(C=5, ca..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=-1, voting='soft',
         weights=[1, 1, 1])

In [18]:
# Prédiction et score du modèle global
predictions = predict(x_test, vectorizer, voting, PREDICT_WORD)
score = accuracy_score(y_test, predictions)
mat_conf = confusion_matrix(y_test, predictions)

print("score voting classifier : {}".format(score))
print("matrice de confusion : \n{}".format(mat_conf))

score voting classifier : 0.30141843971631205
matrice de confusion : 
[[ 3  0  5  8  7  5  6  8]
 [ 0  0  0  0  0  0  1  0]
 [ 4  0 19 14  2  2  3  4]
 [ 8  0  3  9  6  1  7 11]
 [ 3  0  1  0 29 11  0 14]
 [ 0  0  0  0  0  0  0  0]
 [ 2  0  3 13  5  2  9  8]
 [ 1  0  0 13  9  5  2 16]]
